In [ ]:
from xgboost import XGBRegressor
import joblib
import pandas as pd

model = joblib.load('xeno_xgbregressor.pkl')

In [42]:
def cpu(now,choices,kind,model=model):
    scores = {}
    for choice in choices:
        data = {}

        # print(now.keys())
        card_number = now['card_number']
        my_hands = now['my_hands']
        my_played = now['my_played']
        other_played = now['other_played']
        look_hands = now['look_hands']
        looked_hands = now['looked_hands']
        pred_data = now['pred']
        reincarnation = int(now['reincarnation'])

        data['card_number'] = int(card_number)
        for j in range(2):
            if j < len(my_hands):
                data[f'my_hands_{j}'] = int(my_hands[j])
            else:
                data[f'my_hands_{j}'] = 0

        for j in range(15):
            if j < len(my_played):
                data[f'my_played_{j}'] = int(my_played[j])
            else:
                data[f'my_played_{j}'] = 0

        for j in range(len(list(other_played.keys()))):
            for k in range(15):
                if k < len(other_played[j]):
                    data[f'other_played_{j}_{k}'] = int(other_played[j][k])
                else:
                    data[f'other_played_{j}_{k}'] = 0

            for k in range(15):
                if k < len(look_hands[j]):
                    data[f'look_hands_{j}_{k}'] = int(look_hands[j][k])
                else:
                    data[f'look_hands_{j}_{k}'] = 0

            for k in range(15):
                if k < len(looked_hands[j]):
                    data[f'looked_hands_{j}_{k}'] = int(looked_hands[j][k])
                else:
                    data[f'looked_hands_{j}_{k}'] = 0

            for k in range(2):
                if k < len(pred_data[j]):
                    p = pred_data[j][k]
                    data[f'pred_{j}_{k}_object'] = int(p['object'])
                    data[f'pred_{j}_{k}_predcard'] = int(p['pred_card'])
                else:
                    data[f'pred_{j}_{k}_object'] = 0
                    data[f'pred_{j}_{k}_predcard'] = 0
                
                if k < len(pred_data[-1]):
                    p = pred_data[-1][k]
                    data[f'pred_{-1}_{k}_object'] = int(p['object'])
                    data[f'pred_{-1}_{k}_predcard'] = int(p['pred_card'])
                else:
                    data[f'pred_{-1}_{k}_object'] = 0
                    data[f'pred_{-1}_{k}_predcard'] = 0

        data['reincarnation'] = reincarnation
        data['kind'] = kind
        for i in range(10):
            if i < len(choices):
                data[f'choices_{i}'] = choices[i]
            else:
                data[f'choices_{i}'] = 0
        data['choice'] = choice

        df = pd.DataFrame(data=data,index=[0])

        if df.iloc[0]['kind'] == 'draw':
            df.at[0,'kind'] = 0
        elif df.iloc[0]['kind'] == 'opponentChoice':
            df.at[0,'kind'] = 1
        elif df.iloc[0]['kind'] == 'play_card':
            df.at[0,'kind'] = 2
        elif df.iloc[0]['kind'] == 'pred':
            df.at[0,'kind'] = 3
        elif df.iloc[0]['kind'] == 'trush':
            df.at[0,'kind'] = 4

        df.astype(float)
        x = df.values
        score = model.predict(x)
        scores[f'{choice}'] = score
    return max(scores, key=scores.get)

In [45]:
import xeno as x
import random

def choice(now,choices,kind):
    choice = cpu(now=now,choices=choices,kind=kind)
    return choice

def get_name():
    number = random.randint(0,9)
    return 'test_player'+str(number)

game = x.Game(2, get_name=get_name)
data = game.game(choice_func=choice)

if data[0]:
    logs = data[1]
else:
    print(data[1])

In [46]:
for i in range(2):
    choices = logs[i][0]['choices']
    choice = logs[i][0]['choice']
    if logs[i][0]['kind'] == 'draw':
        print('ドロー！')
    elif logs[i][0]['kind'] == 'play_card':
        print('プレイするカードの選択！')
    elif logs[i][0]['kind'] == 'opponentChoice':
        print('対象を選択！')
    elif logs[i][0]['kind'] == 'trush':
        print('相手のカードで捨てさせるカードを選択！')
    print(f'プレイヤー{i}が{choices}から{choice}を選択しました！')

j = [0,0]
while (logs[0][j[0]+1] != 'win' and logs[0][j[0]+1] != 'lose') or (logs[1][j[1]+1] != 'win' and logs[1][j[1]+1] != 'lose'):
    for i in range(2):
        if logs[i][j[i]+1] != 'win' and logs[i][j[i]+1] != 'lose':
            print()
            print(f'プレイヤー{i}のターン！')
            hands = logs[i][j[i]+1]['now']['my_hands']
            print(f'{hands}')
        one = False
        two = False
        opponent_draw = False
        while logs[i][j[i]+1] != 'win' and logs[i][j[i]+1] != 'lose':
            if logs[i][j[i]+1]['kind'] == 'draw':
                if one:
                    two = True
                else:
                    one = True
            if not two:
                j[i] += 1
                choices = logs[i][j[i]]['choices']
                choice = logs[i][j[i]]['choice']
                if logs[i][j[i]]['kind'] == 'draw':
                    one = True
                    print('ドロー！')
                elif logs[i][j[i]]['kind'] == 'play_card':
                    print('プレイするカードの選択！')
                elif logs[i][j[i]]['kind'] == 'opponentChoice':
                    if int(logs[i][j[i]]['now']['my_played'][-1]) == 5 or int(logs[i][j[i]]['now']['my_played'][-1]) == 9:
                        j[(i+1)%2]+=1
                        opponent_choices = logs[(i+1)%2][j[(i+1)%2]]['choices']
                        opponent_choice = logs[(i+1)%2][j[(i+1)%2]]['choice']
                        opponent_draw = True
                    print('対象を選択！')
                elif logs[i][j[i]]['kind'] == 'trush':
                    print('相手のカードで捨てさせるカードを選択！')
                elif logs[i][j[i]]['kind'] == 'pred':
                    print('予測！')
                print(f'プレイヤー{i}が{choices}から{choice}を選択しました！')
                if opponent_draw:
                    print('相手のドロー！')
                    print(f'プレイヤー{(i+1)%2}が{opponent_choices}から{opponent_choice}を選択しました！')

            else:
                print()
                break

for i in range(2):
    print(f'プレイヤー{i}')
    print(logs[i][-1])

ドロー！
プレイヤー0が[5]から5を選択しました！
ドロー！
プレイヤー1が[4]から4を選択しました！

プレイヤー0のターン！
[5]
ドロー！
プレイヤー0が[8]から8を選択しました！
プレイするカードの選択！
プレイヤー0が[5, 8]から5を選択しました！
対象を選択！
プレイヤー0が[0]から0を選択しました！
相手のドロー！
プレイヤー1が[3]から3を選択しました！


プレイヤー1のターン！
[3]
ドロー！
プレイヤー1が[5]から5を選択しました！
プレイするカードの選択！
プレイヤー1が[3, 5]から3を選択しました！
対象を選択！
プレイヤー1が[0]から0を選択しました！


プレイヤー0のターン！
[8]
ドロー！
プレイヤー0が[1]から1を選択しました！
プレイするカードの選択！
プレイヤー0が[8, 1]から1を選択しました！


プレイヤー1のターン！
[5]
ドロー！
プレイヤー1が[8]から8を選択しました！
プレイするカードの選択！
プレイヤー1が[5, 8]から5を選択しました！
対象を選択！
プレイヤー1が[0]から0を選択しました！
相手のドロー！
プレイヤー0が[4]から4を選択しました！


プレイヤー0のターン！
[4]
ドロー！
プレイヤー0が[2]から2を選択しました！
プレイするカードの選択！
プレイヤー0が[4, 2]から2を選択しました！
対象を選択！
プレイヤー0が[0]から0を選択しました！
予測！
プレイヤー0が[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]から10を選択しました！


プレイヤー1のターン！
[8]
ドロー！
プレイヤー1が[3]から3を選択しました！
プレイするカードの選択！
プレイヤー1が[8, 3]から3を選択しました！
対象を選択！
プレイヤー1が[0]から0を選択しました！


プレイヤー0のターン！
[4]
ドロー！
プレイヤー0が[6]から6を選択しました！
プレイするカードの選択！
プレイヤー0が[4, 6]から4を選択しました！


プレイヤー1のターン！
[8]
ドロー！
プレイヤー1が[1]から1を選択しました！
プレイするカードの選択！
プレイヤー1が[8, 1]から1を選択しました！


プレイヤー0のターン！
[6]
ドロー！
